In [8]:
from __future__ import print_function
from __future__ import division

from collections import OrderedDict
import os
import sys
import warnings

import argparse
import logging
import h5py as h5
import numpy as np
import pandas as pd
import scipy.io
from imblearn.under_sampling import OneSidedSelection, NeighbourhoodCleaningRule

import six
from six.moves import range

np.random.seed(1369)

In [3]:
 input_kmer_positive = pd.read_csv('K562_H3K27ac_2000pos.csv', sep=',', header=None, skiprows=1)
 input_kmer_positive = input_kmer_positive.values;
 input_kmer_positive = np.array(input_kmer_positive, dtype = 'int8')
 print(input_kmer_positive.shape)

(8231, 2081)


In [4]:
 input_kmer_negative = pd.read_csv('K562_H3K27ac_2000neg.csv', sep=',', header=None, skiprows=1)
 input_kmer_negative = input_kmer_negative.values
 input_kmer_negative = np.array(input_kmer_negative, dtype = 'int8')
 print(input_kmer_negative.shape)   

(21597, 2081)


In [5]:
 #combining positive and negative sample rowwise
 input_kmer = np.concatenate((input_kmer_positive,input_kmer_negative), axis = 0)
 np.random.shuffle(input_kmer)
 output_label = input_kmer[:,2080]
 input_kmer = np.delete(input_kmer,2080,1)

In [6]:
 X = input_kmer
 y = output_label
 print(X.shape)
 print(y.shape)

(29828, 2080)
(29828,)


In [7]:
 ncl = NeighbourhoodCleaningRule(random_state = 42, return_indices=True)
 X_resampled, y_resampled, idx_resampled = ncl.fit_sample(X, y)
 print(X_resampled.shape)

(23634, 2080)


In [9]:
 h5filename = "histonemodKmer_resample_ncl_K562.h5" 
 h5file = h5.File(h5filename,'a')


In [10]:
 #in_group = h5file.create_group('input')
 h5file.create_dataset('/input/H3K27ac_kmer_2000',data = X_resampled, dtype = np.float64, compression ='gzip')
 #out_group = h5file.create_group('output')
 h5file.create_dataset('/output/H3K27ac_2000',data = y_resampled, dtype = np.int8, compression ='gzip')	
 #h5file.create_dataset('/output/H3K27ac_idx',data = idx_resampled, dtype = np.int8, compression ='gzip')


<HDF5 dataset "H3K27ac_2000": shape (23634,), type "|i1">

In [11]:
 h5file.close()